<a href="https://colab.research.google.com/github/Tobi-KL/mobility_profile_generation/blob/main/Mobility_Profile_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparation

Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
!pip install xlrd
import xlrd
import random

Read Data

In [ ]:
###   Read data    ###
with open('/content/drive/MyDrive/Tobias_MA/Data/data_mop.pkl', 'rb') as input:
        [states_mop,speed_mop,meta_mop,meta_header_mop] = pickle.load(input)

In [ ]:
# nach Jahr 2017 filtern:
meta_mop_filtered = meta_mop[-3820:-1,]

In [ ]:
state_names = ['not known (mobile or not mobile)', 'work', 'work', 'school,training',
               'shopping', 'private', 'bring,pick up', 'freetime', 'home', 
               'else (outside not mobile)', 'not known (not mobile)', 'by foot', 
               'bicycle', 'motorcycle', 'car driver', 'car codriver', 'public transport', 
               'long distance public-transport', 'else (mobile)', 'not known (mobile)']

no_states = int(len(np.unique(states_mop)))
timesteps = int(states_mop.shape[1])

### Classes

In [ ]:
# erstellt ein Haushalt-Objekt
# enthält Informationen über den gewählten Haushalt (Anzahl Bewohner, Anzahl Fahrer, alle Mobilitätsprofile der Bewohner, Position im Datensatz ... )
# vorerst: entweder zufällig oder mit bekannter ID
# TODO: Eingabemöglichkeit verschiedener Parameter (z.B. Haushaltsgröße, Einkommen etc.) -> Rückgabe einer Liste der ähnlichsten Haushalte

class Household:
    def __init__(self, household_ID, randomize_household):
        self.household_ID = household_ID
        #self.household_size = household_size
        self.position = np.where(meta_mop_filtered[:,0] == self.household_ID)[0]


        if randomize_household:
          self.chosen_household = random.choice (meta_mop_filtered.astype(int))
        
        else:
          self.chosen_household = meta_mop_filtered.astype(int) [np.where(meta_mop_filtered[:,0] == self.household_ID)]

        """
        - Constructor of household object
        - householdID: ID of household (MOP-Data) -> sorted in ascending order
        - householdSize: # of occupants of household
        - randomize_household: bool, default=False (defines, whether household should be chosen randomly)
        """
  
    def get_household(self):
      return self.chosen_household.astype(int)
    # returns chosen household (array with >= 1 entries (1 entry = 1 person, 1 year))

    def get_number_of_occupants(self):
      return self.chosen_household[0,5]
    # returns number of occupants living in chosen household
    
    def get_number_of_drivers(self):
      x = self.get_states(0,1008)
      number_of_drivers = 0
      for i in range(0,len(x)):
        if ((x[i]==14).sum() > 0):
          number_of_drivers += 1
      return number_of_drivers
    # returns number of drivers (at least one state == 14) living in chosen household
    
    def get_number_of_cars(self):
      return self.chosen_household[0,7]
    # returns number of cars of chosen household
    
    def get_data_position(self):
      #self.position = np.where(meta_mop[:,0] == self.household_ID)[0]
      position_list =[]
      for i in self.position:
          position_list.append(i)
      return position_list
    # returns indeces of chosen household in data set
    
    def get_states(self, start, end):
      for i in self.position:
        return states_mop[self.position,start:end]
    # returns all states for timespace between start and end

    def get_speeds(self, start, end):
      for i in self.position:
        return speed_mop[self.position,start:end]
    # returns all speeds for timespace between start and end

    def get_persons (self, person_number):
      household_positions = self.get_data_position()
      persons = [Personal_Mobility_Profile (position = i, household_ID = self.household_ID) for i in household_positions]
      return persons[person_number].position
    # returns position of chosen Person-object in data set

In [ ]:
# erstellt das Mobilitätsprofil EINER Person
# dafür muss die position der Person im Datensatz angegeben werden (erste Zeile im Datensatz = Person 0)
# TODO: Methode get_total_distance(). Für die spätere Sortierung der Personen innerhalb eines Haushalts werden Infos zur wöchentlichen Fahrleistung aus der MOP-Tabelle W benötigt

class Personal_Mobility_Profile:
    def __init__(self, position, household_ID):
       self.position = position #position in data
       self.household_ID = household_ID
    
    """
    - Constructor of Personal_Mobility_Profile Object
    - householdID: ID of household (MOP-Data) -> sorted in ascending order
    - position: position of individual person in data set (first line in data set = person 0)
    """

    def get_state(self, start, end):
        states = states_mop[self.position,start:end]
        return states
    # returns states for individual persons between start and end

    def get_speed(self, start, end):
        return speed_mop[self.position,start:end]
    # returns speed for timespace between start and end

    def get_adjusted_states(self, start, end):
        states = self.get_state(start,end)
        states_adjusted = np.where((states == 8) | (states == 14) | (states == 1) | (states == 2), states, 30)
        return states_adjusted
    # returns adjusted states for individual person

    def get_driver_positions(self, start, end):
        list_car_driver = [i for i in range(len(self.get_state(start, end))) if self.get_state(start, end)[i] == 14]
        return list_car_driver
    # returns every position in states where car driver = true
    
    def get_total_distance(self):
      return sum() # aus W-Tabelle
    # returns total distance driven by person in the week

In [ ]:
# wird aufgerufen, um zu prüfen, ob Mobilitätsprofile von Personen im Haushalt zusammengelegt werden müssen (mehr Fahrer im Haushalt, als Autos)
# führt Zusammenlegung, wenn nötig, durch (Personen mit geringster wöchentlicher Fahrleistung werden solange zusammengelegt, bis Anzahl Fahrer = Anzahl Autos)
# gibt die zusammengelegten Mobilitätsprofile aus
# TODO: Methode merge() noch nicht fertig (muss auf get_total_distance zugreifen)

class Profile_Merge:
    def __init__(self, household_ID, number_of_cars, number_of_drivers, drivers):
      self.household_ID = household_ID
      self.number_of_cars = number_of_cars #Anzahl Autos im Haushalt
      self.number_of_drivers = number_of_drivers #Anzahl Personen im Haushalt
      self.drivers = drivers # Liste aller Personen des Haushalts, die mind. einmal Auto als Fahrer (mind. 2) (Liste aller Indezes aus dem Datensatz)

    """
    - Constructor of Profile_Merge Object
    - householdID: ID of household (MOP-Data) -> sorted in ascending order
    - number_of_cars: total number of cars in household
    - number_of_drivers: total number of drivers in household (at least one state = 14)
    - drivers: list of all drivers in household (at least 2)
    """

    def check_merge(self):
      if (self.number_of_drivers > self.number_of_cars):
        print("Merge necessary. Start method merge().")
      else:
        print("No merge necessary.")
    # checks whether a mobility profile merge is necessary
    
    def merge(self):
    # while-Schleife (solange Differenz ungleich 0 -> merge Profile)
    # checke Differenz
    # füge die beiden Fahrer, die am wenigsten fahren, zu einem Profil zusammen
    # Anzahl driver-1
    # checke Differenz usw.
    # gebe die Personen-Profile zurück
      x= self.number_of_drivers
      while x > self.number_of_cars:
        print("merge")
        x -= 1


In [ ]:
# Zuordnung persönliche Mobilitätsprofil(e) zu Auto(s)
# davor muss merge stattgefunden haben -> Anzahl Mobilitätsprofile = Anzahl Autos
# Mobility-Profile mit der höchsten Fahrleistung (MOP W) -> Auto mit höchster Fahrleistung usw.
# Eingabe: alle Mobilitätsprofile der Fahrer, alle Speed-Profile der Fahrer
# Rückgabe: Mobilitätsprofile und Speed-Profile mit zugehöriger Autonummer
# TODO: Methode

class Allocation:
    def __init__(self, mobility_profiles, speed_profiles):
      self.mobility_profiles = mobility_profiles
      self.speed_profiles = speed_profiles
      #self.cars = cars #man weiß automatisch, wie viele Autos verfügbar, da vorher merge

    """
    - Constructor of Allocation Object
    - mobility_profiles: list of mobility profiles of all household-drivers
    - speed_profiles: list of speed profiles of all household-drivers
    """


In [ ]:
# erstellt zu gewünschtem Auto im gewünschten Zeitraum das Mobilitätsprofil
# davor müssen merge und allocation gelaufen sein
# TODO: Methode

class Car_Profile:
    def __init__(self, household_ID, mobility_profile, speed_profile, start, end):
      self.household_ID = household_ID
      self.mobility_profile = mobility_profile # zum Auto gehöriges Mobility-Profile
      self.speed_profile = speed_profile # zum Auto gehöriges Speed-Profile
      self.start = start
      self.end = end
    
    """
    - Constructor of Car-Profile Object
    - householdID: ID of household (MOP-Data) -> sorted in ascending order
    - mobility_profile: mobility profile of car
    - speed_profile: speed profile of car
    - start: start timestep
    - end: end timestep
    """
    
    def get_car_profile_plot(self):
      #print("Profile for the chosen car from household " + self.household_ID + " for timesteps " + self.start + " until " + self.end + " :")
      return plt.plot(self.mobility_profile)
    
    


In [ ]:
#class Car:
    def __init__(self):
       #self.consumption = consumption 
       #self.household_ID = household_ID
       #self.segment = segment
       


In [ ]:
#class Route:
    def __init__(self, household_ID, route_ID, departure_time, departure_point, arrival_time, arrival_point, purpose, length):
        self.household_ID = household_ID
        self.route_ID = route_ID
        self.departure_time = departure_time  # in Minuten seit Wochenbeginn 
        self.departure_point = departure_point
        self.arrival_time = arrival_time
        self.arrival_point = arrival_point # letzte Fahrt der Woche endet immer zuhause
        self.purpose = purpose
        self.length = length

#car_ID -> Zuordnung
#-> MOP W

### Tool

Erstellung eines Haushalt-Objektes:

*   Bsp. für HH mit 1 Person: 4301011562;
*   Bsp. für HH mit mehreren Personen: 4711036120



In [ ]:
# Erstellung eines Haushalt-Objekts in meta_mop
# randomize ermöglicht zufälligen Haushalt
# ansonsten: gewünschte ID eingeben

household_1 =  Household(household_ID = 4301012233, randomize_household=False)
household_1.get_household()

In [ ]:
# Anzahl Bewohner des Haushalts
# Anzahl Fahrer im Haushalt
# Anzahl Autos im Haushalt

print('number of occupants in chosen household: ', household_1.get_number_of_occupants())
print('number of drivers in chosen household: ', household_1.get_number_of_drivers())
print('number of cars in chosen household: ', household_1.get_number_of_cars())

In [ ]:
# Informationen über die Position des Haushalts bzw. der Personen im Datensatz (filtered: nur 2017)
# gibt die Position der gewünschten Person im Datensatz an. (0 = erste Person des HH usw.)

print('The household can be found in the data set at row(s): ', household_1.get_data_position())

person_number = 0     # 0: first person, 1: second person ...
print('Person', person_number, 'can be found in the data set at row: ', household_1.get_persons(person_number))

In [ ]:
# Ausgabe der Mobilitätsprofile

start = 0     # timesteps
end = 1008
person_number = 0

mobility_profiles = household_1.get_states(start,end)

print('Mobility profile of person', person_number, ':')
plt.plot(mobility_profiles[person_number])

In [ ]:
# Ausgabe der Speed-Profiles

start = 0     # timesteps
end = 1008
person_number = 0

speed_profiles = household_1.get_speeds(start, end)

print('Speed profile of person', person_number, ':')
plt.plot(speed_profiles[person_number])

In [ ]:
# Ausgabe der States eines individuellen HH-Mitglieds
person_number = 0
household_ID = 4301012233
start = 0
end = 504

# erstelle Personal_Mobility_Profile Objekt:
person_1 = Personal_Mobility_Profile (position = household_1.get_persons(person_number), household_ID = household_ID)
print('original states for person', person_number,':\n', person_1.get_state(start,end))
print('\n')

# setze alle states außer 1,2 (work), 8 (home), 14 (car driver) auf 30
print('adjusted states for person', person_number, ':\n', person_1.get_adjusted_states(start, end))
print('\n')

# Liste aller Positionen "Car Driver":
print('positions where car driver is true:\n', person_1.get_driver_positions(start, end))

### Test

In [ ]:
# Fall: 1 Fahrzeug, mehrere Personen
## Prüfungslogik, ob alle Fahrten nur einem Auto zugeordnet werden können
# alle Arrays abgleichen, ob nie gleichzeitig eine 14 steht
# wenn true -> Fehler gefunden

result = [(x == y == 14) for x,y in zip(household_1.get_states(0,1008)[0], household_1.get_states(0,1008) [1])]
np.any(result)

In [ ]:
# Fahrzeugprofil für das Auto erstellen:
# 3 Personen: 20044, 20045, 20046
# nimmt immer den maximalen Wert
vehicleProfile = mobProfile1

vehicleProfile = np.maximum.reduce([mobProfile1, mobProfile2])

print(vehicleProfile)
print(mobProfile1)

In [ ]:
# Fall 1 Auto, 1 Person (Auto hat das gleiche Mobilitätsprofil, wie die Person)
      if (household.get_number_of_occupants() == 1 and household.get_number_of_cars() == 1):
        print("Car profile for household with 1 Car and 1 Person.")
        #create Person-Object
        #person = Person(0, household_ID)
        #person.get_state(start, end)
        #household.get_persons(0).get_states(start,end)

      
      # Fall 1 Auto, mehrere Personen (Personen solange zusammenlegen, bis Anzahl gleich ist)
      if (household.get_number_of_occupants() > 1 and household.get_number_of_cars() == 1):
        print("Car profile for household with 1 Car and > 1 Person.")
      
      # Fall mehrere Autos, gleich viele Personen
      if (household.get_number_of_cars() > 1 and  household.get_number_of_cars() == household.get_number_of_occupants()):
        print("Car profile for household with >1 Car and the same amount of persons.")

      # Fall mehrere Autos, mehr Personen als Autos (Personen so lange zusammenlegen, bis Anzahl gleich ist)
      if (household.get_number_of_cars() > 1 and  household.get_number_of_cars() > household.get_number_of_occupants()):
        print("Car profile for household with >1 Car and more persons than cars.")

In [ ]:
test = Profile_Merge(1, 2,4,7)
test.merge()

In [ ]:
test_car_profile = Car_Profile (household_ID = 4711036120,  mobility_profile = household_1.get_states(0,1008)[0], speed_profile = household_1.get_speeds(0,1008)[0], start=0, end=1008)
test_car_profile.get_car_profile_plot()

In [ ]:
car_profile_1.get_profile()